In [7]:
import warnings 
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import seaborn as sns
import matplotlib.pyplot as plt 
warnings.filterwarnings("ignore")

In [2]:
from scripts.data_clean import clean_data
from scripts.data_read import read_data
from scripts.normalization import normal

In [3]:
df = read_data("../data/cleaned_data.xlsx")

In [5]:
df =  df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [6]:
df

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,13114483460844900352,2019-04-04 12:01:18,770,2019-04-25 14:35:31,662,86399.000000,2.082014e+14,33664962239,3.552121e+13,9164566995485190,...,1.585461e+07,2.501332e+06,8.198936e+06,9.656251e+06,2.780823e+08,1.434415e+07,1.717444e+08,8.814393e+06,36749741,308879636.0
1,13114483482878900224,2019-04-09 13:04:04,235,2019-04-25 08:15:48,606,86399.000000,2.082019e+14,33681854413,3.579401e+13,L77566A,...,2.024740e+07,1.911173e+07,1.833841e+07,1.722713e+07,6.087501e+08,1.170709e+06,5.269042e+08,1.505514e+07,53800391,653384965.0
2,13114483484080500736,2019-04-09 17:42:11,1,2019-04-25 11:58:13,652,86399.000000,2.082003e+14,33760627129,3.528151e+13,D42335A,...,1.972566e+07,1.469958e+07,1.758779e+07,6.163408e+06,2.295846e+08,3.956300e+05,4.106926e+08,4.215763e+06,27883638,279807335.0
3,13114483485442799616,2019-04-10 00:31:25,486,2019-04-25 07:36:35,171,86399.000000,2.082014e+14,33750343200,3.535661e+13,T21824A,...,2.138812e+07,1.514664e+07,1.399465e+07,1.097942e+06,7.995382e+08,1.084972e+07,7.490399e+08,1.279728e+07,43324218,846028530.0
4,13114483499480700928,2019-04-12 20:10:23,565,2019-04-25 10:40:32,954,86399.000000,2.082014e+14,33699795932,3.540701e+13,D88865A,...,1.525938e+07,1.896287e+07,1.712458e+07,4.152180e+05,5.277072e+08,3.529801e+06,5.507095e+08,1.391032e+07,38542814,569138589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7277825670196679680,2019-04-29 07:28:42,451,2019-04-30 06:02:33,214,81230.000000,2.082015e+14,33650688697,3.548311e+13,D20434A,...,1.619167e+07,1.176343e+07,1.788370e+07,1.967816e+07,5.266097e+08,9.197207e+06,3.264510e+06,1.348742e+07,57628851,574175259.0
149997,7349883264234609664,2019-04-29 07:28:42,483,2019-04-30 10:41:33,187,97970.000000,2.082019e+14,33663449963,3.566051e+13,D10223C,...,1.387723e+07,8.288284e+06,1.935015e+07,2.129315e+07,6.268931e+08,4.735033e+06,7.121804e+08,2.457758e+06,39135081,666648844.0
149998,13114483573367300096,2019-04-29 07:28:43,283,2019-04-30 10:46:12,810,98249.000000,2.082017e+14,33621890103,3.572121e+13,T51102A,...,2.266051e+07,1.855903e+06,9.963942e+06,5.065760e+06,5.535395e+08,1.339432e+07,1.211009e+08,1.131473e+07,34912224,592786405.0
149999,13114483573367300096,2019-04-29 07:28:43,696,2019-04-30 10:40:34,327,97910.000000,2.082015e+14,33619622058,8.618620e+13,L88342B,...,8.817106e+06,8.305402e+06,3.322253e+06,1.317259e+07,3.525370e+08,2.529475e+06,8.147131e+08,1.406930e+06,29626096,371895920.0


In [23]:
df.isna().sum()

Bearer Id                         0
Start                             0
Start ms                          0
End                               0
End ms                            0
Dur. (ms)                         0
IMSI                              0
MSISDN/Number                     0
IMEI                              0
Last Location Name                0
Avg RTT DL (ms)                   0
Avg RTT UL (ms)                   0
Avg Bearer TP DL (kbps)           0
Avg Bearer TP UL (kbps)           0
DL TP < 50 Kbps (%)               0
50 Kbps < DL TP < 250 Kbps (%)    0
250 Kbps < DL TP < 1 Mbps (%)     0
DL TP > 1 Mbps (%)                0
UL TP < 10 Kbps (%)               0
10 Kbps < UL TP < 50 Kbps (%)     0
50 Kbps < UL TP < 300 Kbps (%)    0
UL TP > 300 Kbps (%)              0
Activity Duration DL (ms)         0
Activity Duration UL (ms)         0
Dur. (ms).1                       0
Handset Manufacturer              0
Handset Type                      0
Nb of sec with Vol DL < 6250

#### data_aggregation

In [35]:
#slicing the useful column
df_engag_metrics = df.iloc[:,np.r_[0,5,7,43:45]]

In [36]:
#create column of the total data 
df_engag_metrics['total'] = df_engag_metrics['Total UL (Bytes)'] + df_engag_metrics['Total DL (Bytes)']

In [37]:
df_engag_metrics.head()

,Bearer Id,Dur. (ms),MSISDN/Number,Total UL (Bytes),Total DL (Bytes),total
0,13114483460844900352,86399.0,33664962239,36749741,308879636.0,345629377.0
1,13114483482878900224,86399.0,33681854413,53800391,653384965.0,707185356.0
2,13114483484080500736,86399.0,33760627129,27883638,279807335.0,307690973.0
3,13114483485442799616,86399.0,33750343200,43324218,846028530.0,889352748.0
4,13114483499480700928,86399.0,33699795932,38542814,569138589.0,607681403.0


###### aggregate by the session frequency 

In [44]:
df_engag_metrics.iloc[:,np.r_[0,2]].groupby('MSISDN/Number').agg({'Bearer Id':'count'}).reset_index().sort_values(by='Bearer Id',ascending=False).head(10).reset_index().drop('index',axis=1)

,MSISDN/Number,Bearer Id
0,33663797508,7501
1,33626320676,18
2,33614892860,17
3,33625779332,17
4,33659725664,16
5,33760536639,15
6,33675877202,15
7,33667163239,13
8,33760413819,12
9,33627080969,12


###### aggregation by duration of session

In [46]:
df_engag_metrics.iloc[:,np.r_[2,5]].groupby('MSISDN/Number').agg({'total':'sum'}).reset_index().sort_values(by='total',ascending=False).head(10).reset_index().drop('index',axis=1)

,MSISDN/Number,total
0,33663797508,3.700910e+12
1,33614892860,8.826326e+09
2,33760536639,8.495067e+09
3,33625779332,8.474657e+09
4,33626320676,7.951348e+09
5,33675877202,7.891111e+09
6,33659725664,7.680848e+09
7,33666464084,7.308501e+09
8,33760413819,7.112835e+09
9,33664712899,6.852224e+09


###### aggregation by total data 

In [47]:
df_engag_metrics.iloc[:,np.r_[1,2]].groupby('MSISDN/Number').agg({'Dur. (ms)':'sum'}).reset_index().sort_values(by='Dur. (ms)',ascending=False).head(10).reset_index().drop('index',axis=1)

,MSISDN/Number,Dur. (ms)
0,33663797508,6.491735e+08
1,33659725664,1.905478e+06
2,33626320676,1.785419e+06
3,33659359429,1.747994e+06
4,33675877202,1.745131e+06
5,33614892860,1.678397e+06
6,33760536639,1.651084e+06
7,33625779332,1.599052e+06
8,33664712899,1.334982e+06
9,33606582008,1.322251e+06


In [50]:
# engagement metrics 
df3 = df_engag_metrics.iloc[:,np.r_[0,1,5]]

In [52]:
df3.head()

,Bearer Id,Dur. (ms),total
0,13114483460844900352,86399.0,345629377.0
1,13114483482878900224,86399.0,707185356.0
2,13114483484080500736,86399.0,307690973.0
3,13114483485442799616,86399.0,889352748.0
4,13114483499480700928,86399.0,607681403.0


In [53]:
#normalize 
